In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback  #early stopping ta aktu pore implement korchi
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
import keras

Using TensorFlow backend.


In [ ]:
X , Y = loadData()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [2]:
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return


In [21]:
def base_model():
        model = Sequential()
        model.add(Dense(250, input_dim=10,
                        kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        
        model.add(Dense(125, kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        
        model.add(Dense(85, kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        
        model.add(Dense(35, kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.1))
        
        model.add(Dense(10, kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
#         model.add(Dropout(0.08))
        model.add(Dense(1, activation='sigmoid'))

        # Compile model
        opt = keras.optimizers.adam(lr=0.01, decay=1e-7)
        model.compile(optimizer=opt, metrics = ['accuracy'], loss='binary_crossentropy')

        return model

In [22]:
model = base_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 250)               2750      
_________________________________________________________________
activation_46 (Activation)   (None, 250)               0         
_________________________________________________________________
batch_normalization_46 (Batc (None, 250)               1000      
_________________________________________________________________
dropout_37 (Dropout)         (None, 250)               0         
_________________________________________________________________
dense_56 (Dense)             (None, 125)               31375     
_________________________________________________________________
activation_47 (Activation)   (None, 125)               0         
_________________________________________________________________
batch_normalization_47 (Batc (None, 125)               500       
__________

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, 
          callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test))])